In [0]:
'''
A RNN implementation example using TensorFlow..
Next word prediction after n_input words learned from text file.
A story is automatically generated if the predicted word is fed back as input.
'''
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
import random
import collections
import time
from google.colab import files
files.upload()

Saving omni.txt to omni.txt


{'omni.txt': b'"They\'re made out of meat."\r\n\r\n     "Meat?"\r\n\r\n     "Meat. They\'re made out of meat."\r\n\r\n     "Meat?"\r\n\r\n     "There\'s no doubt about it. We picked up several from different parts of the planet, took them aboard our recon vessels, and probed them all the way through. They\'re completely meat."\r\n\r\n     "That\'s impossible. What about the radio signals? The messages to the stars?"\r\n\r\n     "They use the radio waves to talk, but the signals don\'t come from them. The signals come from machines."\r\n\r\n     "So who made the machines? That\'s who we want to contact."\r\n\r\n     "They made the machines. That\'s what I\'m trying to tell you. Meat made the machines."\r\n\r\n     "That\'s ridiculous. How can meat make a machine? You\'re asking me to believe in sentient meat."\r\n\r\n     "I\'m not asking you, I\'m telling you. These creatures are the only sentient race in that sector and they\'re made out of meat."photomaxmix\r\n\r\n     "Maybe they\'r

In [0]:
start_time = time.time()
def elapsed(sec):
    if sec<60:
        return str(sec) + " sec"
    elif sec<(60*60):
        return str(sec/60) + " min"
    else:
        return str(sec/(60*60)) + " hr"

In [0]:
# Text file containing words for training
training_file = 'omni.txt'

def read_data(fname):
    with open(fname) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    content = [word for i in range(len(content)) for word in content[i].split()]
    content = np.array(content)
    return content

training_data = read_data(training_file)
print("Loaded training data...")

Loaded training data...


In [0]:
logs_path = '/tmp/tensorflow/rnn_words'
writer = tf.summary.FileWriter(logs_path)

In [0]:
training_data

array(['"They\'re', 'made', 'out', 'of', 'meat."', '"Meat?"', '"Meat.',
       "They're", 'made', 'out', 'of', 'meat."', '"Meat?"', '"There\'s',
       'no', 'doubt', 'about', 'it.', 'We', 'picked', 'up', 'several',
       'from', 'different', 'parts', 'of', 'the', 'planet,', 'took',
       'them', 'aboard', 'our', 'recon', 'vessels,', 'and', 'probed',
       'them', 'all', 'the', 'way', 'through.', "They're", 'completely',
       'meat."', '"That\'s', 'impossible.', 'What', 'about', 'the',
       'radio', 'signals?', 'The', 'messages', 'to', 'the', 'stars?"',
       '"They', 'use', 'the', 'radio', 'waves', 'to', 'talk,', 'but',
       'the', 'signals', "don't", 'come', 'from', 'them.', 'The',
       'signals', 'come', 'from', 'machines."', '"So', 'who', 'made',
       'the', 'machines?', "That's", 'who', 'we', 'want', 'to',
       'contact."', '"They', 'made', 'the', 'machines.', "That's", 'what',
       "I'm", 'trying', 'to', 'tell', 'you.', 'Meat', 'made', 'the',
       'machines."'

In [0]:
def build_dataset(words):
    count = collections.Counter(words).most_common()
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return dictionary, reverse_dictionary

In [0]:
def RNN(x, weights, biases):

    # reshape to [1, n_input]
    x = tf.reshape(x, [-1, n_input])

    # Generate a n_input-element sequence of inputs
    # (eg. [had] [a] [general] -> [20] [6] [33])
    x = tf.split(x,n_input,1)

    # 2-layer LSTM, each layer has n_hidden units.
    # Average Accuracy= 95.20% at 50k iter
    rnn_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(n_hidden),rnn.BasicLSTMCell(n_hidden)])

    # 1-layer LSTM with n_hidden units but with lower accuracy.
    # Average Accuracy= 90.60% 50k iter
    # Uncomment line below to test but comment out the 2-layer rnn.MultiRNNCell above
    # rnn_cell = rnn.BasicLSTMCell(n_hidden)

    # generate prediction
    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)

    # there are n_input outputs but
    # we only want the last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [0]:
dictionary, reverse_dictionary = build_dataset(training_data)
vocab_size = len(dictionary)

In [0]:
# Parameters
learning_rate = 0.05
training_iters = 50000
display_step = 1000
n_input = 3

# number of units in RNN cell
n_hidden = 512

# tf Graph input
x = tf.placeholder("float", [None, n_input, 1])
y = tf.placeholder("float", [None, vocab_size])

# RNN output node weights and biases
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, vocab_size]))
}
biases = {
    'out': tf.Variable(tf.random_normal([vocab_size]))
}

In [0]:
pred = RNN(x, weights, biases)

# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

# Model evaluation
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:
# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as session:
    session.run(init)
    step = 0
    offset = random.randint(0,n_input+1)
    end_offset = n_input + 1
    acc_total = 0
    loss_total = 0

    writer.add_graph(session.graph)

    while step < training_iters:
        # Generate a minibatch. Add some randomness on selection process.
        if offset > (len(training_data)-end_offset):
            offset = random.randint(0, n_input+1)

        symbols_in_keys = [ [dictionary[ str(training_data[i])]] for i in range(offset, offset+n_input) ]
        symbols_in_keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])

        symbols_out_onehot = np.zeros([vocab_size], dtype=float)
        symbols_out_onehot[dictionary[str(training_data[offset+n_input])]] = 1.0
        symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])

        _, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, pred], \
                                                feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
        loss_total += loss
        acc_total += acc
        if (step+1) % display_step == 0:
            print("Iter= " + str(step+1) + ", Average Loss= " + \
                  "{:.6f}".format(loss_total/display_step) + ", Average Accuracy= " + \
                  "{:.2f}%".format(100*acc_total/display_step))
            acc_total = 0
            loss_total = 0
            symbols_in = [training_data[i] for i in range(offset, offset + n_input)]
            symbols_out = training_data[offset + n_input]
            symbols_out_pred = reverse_dictionary[int(tf.argmax(onehot_pred, 1).eval())]
            print("%s - [%s] vs [%s]" % (symbols_in,symbols_out,symbols_out_pred))
        step += 1
        offset += (n_input+1)
    print("Optimization Finished!")
    print("Elapsed time: ", elapsed(time.time() - start_time))
    
    
    while True:
        prompt = "%s words: " % n_input
        sentence = input(prompt)
        sentence = sentence.strip()
        words = sentence.split(' ')
        if len(words) != n_input:
            continue
        try:
            symbols_in_keys = [dictionary[str(words[i])] for i in range(len(words))]
            for i in range(32):
                keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])
                onehot_pred = session.run(pred, feed_dict={x: keys})
                onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval())
                sentence = "%s %s" % (sentence,reverse_dictionary[onehot_pred_index])
                symbols_in_keys = symbols_in_keys[1:]
                symbols_in_keys.append(onehot_pred_index)
            print(sentence)
        except:
            print("Word not in dictionary")


Iter= 1000, Average Loss= 20.605031, Average Accuracy= 1.20%
['Any', 'others?', 'Anyone'] - [interesting] vs [closed.]
Iter= 2000, Average Loss= 22.708942, Average Accuracy= 0.90%
["You're", 'sure', 'they'] - [won't] vs [probed?]
Iter= 3000, Average Loss= 22.869608, Average Accuracy= 0.70%
['them', 'to', 'the'] - [speed] vs [limits]
Iter= 4000, Average Loss= 22.729935, Average Accuracy= 0.80%
['want', 'to', 'make'] - [contact] vs [really]
Iter= 5000, Average Loss= 22.505232, Average Accuracy= 0.80%
['are', 'required', 'to'] - [contact,] vs [we]
Iter= 6000, Average Loss= 22.572218, Average Accuracy= 0.80%
['do', 'you', 'think'] - [is] vs [what]
Iter= 7000, Average Loss= 22.387144, Average Accuracy= 0.80%
['to', 'talk', 'to'] - [meat."] vs [supposed]
Iter= 8000, Average Loss= 22.285392, Average Accuracy= 0.70%
['Finally.', 'Yes.', 'They'] - [are] vs [you.]
Iter= 9000, Average Loss= 22.271237, Average Accuracy= 0.80%
['telling', 'you.', 'The'] - [brain] vs [I'm]
Iter= 10000, Average Loss=

KeyboardInterrupt: ignored